# Introduction
## Goal.
The goal of this lab is to investigate, by means of agent-based simulations, an example
of Evolutionary Robotics. In particular, we will focus on a maze navigation robot task, and see
how Evolutionary Algorithms can be used to solve this problem.

Note once again that, unless otherwise specified, in this module’s exercises we will use real-valued
genotypes and that the aim of the algorithms will be to minimize the fitness function $f(x)$, i.e.
lower values correspond to a better fitness!

## Helper code


In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np


class NN:
    def __init__(self, nodes: list[int]):
        self.nodes = nodes
        self.activations: list[list[float]] = [
            [0 for _ in range(node)] for node in nodes
        ]
        self.nweights = sum([
            self.nodes[i] * self.nodes[i + 1] for i in range(len(self.nodes) - 1)
        ])  # nodes[0]*nodes[1]+nodes[1]*nodes[2]+nodes[2]*nodes[3]

        self.weights = [[] for _ in range(len(self.nodes) - 1)]

    def activate(self, inputs: list[float]):
        self.activations[0] = [np.tanh(inputs[i]) for i in range(self.nodes[0])]
        for i in range(1, len(self.nodes)):
            self.activations[i] = [0.0 for _ in range(self.nodes[i])]
            for j in range(self.nodes[i]):
                sum = 0  # self.weights[i - 1][j][0]
                for k in range(self.nodes[i - 1]):
                    sum += self.activations[i - 1][k - 1] * self.weights[i - 1][j][k]
                self.activations[i][j] = np.tanh(sum)
        return np.array(self.activations[-1])

    def set_weights(self, weights: list[float]):
        # self.weights = [[] for _ in range(len(self.nodes) - 1)]
        c = 0
        for i in range(1, len(self.nodes)):
            self.weights[i - 1] = [
                [0 for _ in range(self.nodes[i - 1])] for __ in range(self.nodes[i])
            ]
            for j in range(self.nodes[i]):
                for k in range(self.nodes[i - 1]):
                    self.weights[i - 1][j][k] = weights[c]
                    c += 1
        # print(c)

    def get_list_weights(self) -> list[float]:
        wghts = []
        for i in range(1, len(self.nodes)):
            for j in range(self.nodes[i]):
                for k in range(self.nodes[i - 1]):
                    wghts.append(np.abs(self.weights[i - 1][j][k]))
        return wghts

In [ ]:
import math
from typing import Any, Optional

import pygame
from pygame import Surface
import time

WIDTH = 1920
HEIGHT = 1080

CAR_SIZE_X = 30
CAR_SIZE_Y = 30

BORDER_COLOR = (0, 0, 0, 255)  # Color To Crash on Hit
OBJ_COLOR = (45, 157, 10, 255)
current_generation = 0  # Generation counter
pygame.init()
bck_driver = pygame.display.get_driver()
pygame.quit()


class Car:
    def __init__(
        self, pos: tuple[float, float] = (200, 900), carSprite: str = "img/car.png"
    ):
        # Load Car Sprite and Rotate
        self.sprite = pygame.image.load(carSprite).convert()  # Convert Speeds Up A Lot
        self.sprite = pygame.transform.scale(self.sprite, (CAR_SIZE_X, CAR_SIZE_Y))
        self.rotated_sprite = self.sprite

        # self.position = [690, 740] # Starting Position
        self.position = list(pos)  # [200, 900]  # Starting Position
        self.angle = 0
        self.speed = 0

        self.speed_set = False  # Flag For Default Speed Later on

        self.center = [
            self.position[0] + CAR_SIZE_X / 2,
            self.position[1] + CAR_SIZE_Y / 2,
        ]  # Calculate Center

        self.radars = []  # List For Sensors / Radars
        self.drawing_radars = []  # Radars To Be Drawn

        self.collision = False  # Boolean To Check If Car is Crashed

        self.distance = 0  # Distance Driven
        self.time = 0  # Time Passed
        self.times = 0.0

    def draw(self, screen: Surface):
        screen.blit(self.rotated_sprite, self.position)  # Draw Sprite
        self.draw_radar(screen)  # OPTIONAL FOR SENSORS

    def draw_radar(self, screen: Surface):
        # Optionally Draw All Sensors / Radars
        for radar in self.radars:
            position = radar[0]
            pygame.draw.line(screen, (0, 255, 0), self.center, position, 1)
            pygame.draw.circle(screen, (0, 255, 0), position, 5)

    def check_collision(self, game_map: Surface):
        self.collision = False
        for point in self.corners:
            # If Any Corner Touches Border Color -> Crash
            # Assumes Rectangle
            if 0 <= point[0] < WIDTH and 0 <= point[1] < HEIGHT:
                if game_map.get_at((int(point[0]), int(point[1]))) == BORDER_COLOR:  # type: ignore
                    self.collision = True

                    break
            else:
                self.collision = True
                break

    def check_radar(
        self, degree: float, game_map: Surface, otherCar: Optional[list[int]] = None
    ):
        length = 0
        x = int(
            self.center[0]
            + math.cos(math.radians(360 - (self.angle + degree))) * length
        )
        y = int(
            self.center[1]
            + math.sin(math.radians(360 - (self.angle + degree))) * length
        )
        # print(x)
        # print(y)
        # print(game_map.get_size())
        # While We Don't Hit BORDER_COLOR AND length < 300 (just a max) -> go further and further

        # s
        color = game_map.get_at((x, y))
        cosl = math.cos(0.0174533 * (360 - (self.angle + degree)))
        senl = math.sin(0.0174533 * (360 - (self.angle + degree)))
        # distance_other_car = self.calc_distance([x,y], otherCar) if otherCar is not None else np.inf or distance_other_car <= 15
        while not (color == BORDER_COLOR or color == OBJ_COLOR) and length < 100:  # type: ignore
            length = length + 1
            x = int(self.center[0] + cosl * length)
            y = int(self.center[1] + senl * length)
            # distance_other_car = self.calc_distance([x, y], otherCar) if otherCar is not None else np.inf
            color = game_map.get_at((x, y))

        # Calculate Distance To Border And Append To Radars List
        dist = int(
            math.sqrt(math.pow(x - self.center[0], 2) + math.pow(y - self.center[1], 2))
        )
        self.radars.append([(x, y), dist])

    def _inner_update(self, game_map: Surface, otherCar: Optional[list[int]] = None):
        if not self.speed_set:
            self.speed = 20
            self.speed_set = True

        # Get Rotated Sprite And Move Into The Right X-Direction
        # Don't Let The Car Go Closer Than 20px To The Edge
        self.rotated_sprite = self.rotate_center(self.sprite, self.angle)

        self.position[0] += math.cos(math.radians(360 - self.angle)) * self.speed
        self.position[0] = max(self.position[0], 120)
        self.position[0] = min(self.position[0], WIDTH - 120)

        # Increase Distance and Time
        self.distance += self.speed
        self.time += 1

        # Same For Y-Position
        self.position[1] += math.sin(math.radians(360 - self.angle)) * self.speed
        self.position[1] = max(self.position[1], 90)
        self.position[1] = min(self.position[1], WIDTH - 90)

        # Calculate New Center
        self.center = [
            int(self.position[0]) + CAR_SIZE_X / 2,
            int(self.position[1]) + CAR_SIZE_Y / 2,
        ]

        # Calculate Four Corners
        # Length Is Half The Side
        length = 0.5 * CAR_SIZE_X
        left_top = [
            self.center[0] + math.cos(math.radians(360 - (self.angle + 30))) * length,
            self.center[1] + math.sin(math.radians(360 - (self.angle + 30))) * length,
        ]
        right_top = [
            self.center[0] + math.cos(math.radians(360 - (self.angle + 150))) * length,
            self.center[1] + math.sin(math.radians(360 - (self.angle + 150))) * length,
        ]
        left_bottom = [
            self.center[0] + math.cos(math.radians(360 - (self.angle + 210))) * length,
            self.center[1] + math.sin(math.radians(360 - (self.angle + 210))) * length,
        ]
        right_bottom = [
            self.center[0] + math.cos(math.radians(360 - (self.angle + 330))) * length,
            self.center[1] + math.sin(math.radians(360 - (self.angle + 330))) * length,
        ]
        self.corners = [left_top, right_top, left_bottom, right_bottom]

        # Check Collisions And Clear Radars

        self.check_collision(game_map)
        self.radars.clear()
        # ll = time.time()
        # From -90 To 120 With Step-Size 45 Check Radar
        for d in range(0, 360, 90):
            self.check_radar(d, game_map, otherCar)
        # self.times = time.time() - ll

    def update(self, game_map: Surface, otherCar: Optional[list[int]] = None):
        # Set The Speed To 20 For The First Time
        # Only When Having 4 Output Nodes With Speed Up and Down

        if not self.collision:
            self._inner_update(game_map, otherCar)

        else:  # calculate new position if blocked not update else update
            self.times = 0
            newPosition: list[float] = [0, 0]
            newPosition[0] += int(math.cos(math.radians(360 - self.angle)) * self.speed)
            newPosition[0] = max(self.position[0], 20)
            newPosition[0] = min(self.position[0], WIDTH - 120)
            # Same For Y-Position
            newPosition[1] += int(math.sin(math.radians(360 - self.angle)) * self.speed)
            newPosition[1] = max(self.position[1], 90)
            newPosition[1] = min(self.position[1], HEIGHT - 90)
            x = int(newPosition[0])
            y = int(newPosition[1])

            if not game_map.get_at((x, y)) == BORDER_COLOR:  # type: ignore
                self._inner_update(game_map, otherCar)

        # print(self.times[-1])

    def get_data(self) -> list[float]:
        # Get Distances To Border
        radars = self.radars
        return_values: list[float] = [0, 0, 0, 0]
        for i, radar in enumerate(radars):
            return_values[i] = radar[1] / 100

        return return_values

    def is_collided(self) -> bool:
        # Basic Alive Function
        return self.collision

    def get_reward(self) -> float:
        # Calculate Reward (Maybe Change?)
        # return self.distance / 50.0
        return self.distance / (CAR_SIZE_X / 2)

    def rotate_center(self, image: Surface, angle: float) -> Surface:
        # Rotate The Rectangle
        rectangle = image.get_rect()
        rotated_image = pygame.transform.rotate(image, angle)
        rotated_rectangle = rectangle.copy()
        rotated_rectangle.center = rotated_image.get_rect().center
        rotated_image = rotated_image.subsurface(rotated_rectangle).copy()
        return rotated_image


def run_simulation(
    agents: list[Any], map: str = "white", render: bool = False
) -> tuple[list[float], list[list[float]]]:
    # Initialize PyGame And The Display
    # init = time.time()
    if not render:
        os.environ["SDL_VIDEODRIVER"] = "dummy"
    else:
        os.environ["SDL_VIDEODRIVER"] = bck_driver
        if os.environ["SDL_VIDEODRIVER"] == "dummy":
            os.environ.pop("SDL_VIDEODRIVER", None)

    pygame.init()
    screen = None
    if render:
        screen = pygame.display.set_mode(
            (WIDTH, HEIGHT), pygame.FULLSCREEN | pygame.SCALED
        )

    else:
        screen = pygame.display.set_mode((WIDTH, HEIGHT))

    # Clock Settings
    # Font Settings & Loading Map

    clock = pygame.time.Clock()

    clock.tick(10 if render else 100000)
    game_map = pygame.image.load(map).convert()  # Convert Speeds Up A Lot
    # print("Init "+str(time.time()-init))
    global current_generation
    current_generation += 1

    # Simple Counter To Roughly Limit Time (Not Good Practice)
    # counter = 0
    cars = [Car() for _ in range(len(agents))]
    # fitness = 0
    obs = [[] for _ in range(len(agents))]
    # ss = time.time()
    # print("ffff "+str(len(cars)))
    ups = []
    timess = [[], [], []]
    for _ in range(250):
        # Exit On Quit Event
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit(0)

        # For Each Car Get The Acton It Takes
        # acts = time.time()
        for i in range(len(agents)):
            # print("cars "+str(i)+"  "+str(cars[i]))
            outtime = time.time()
            ob = cars[i].get_data()
            pos = [cars[i].center[0] / WIDTH, cars[i].center[1] / HEIGHT]
            distance_norm = math.sqrt(
                ((1730 / WIDTH) - pos[0]) ** 2 + ((165 / HEIGHT) - pos[1]) ** 2
            )
            bearing = (
                np.arctan2(165 - cars[i].center[1], 1730 - cars[i].center[0]) / np.pi
            )

            ob = [distance_norm, bearing, *ob]
            obs[i].append(ob)

            # s = time.time()
            output = agents[i].activate(ob)

            choice = np.argmax(output)  # random.randint(0,4)#
            if choice == 0:
                cars[i].angle += 5  # Left
            elif choice == 1:
                cars[i].angle -= 5  # Right
            elif choice == 2:
                cars[i].speed = -10  # Slow Down
            elif choice == 3:
                cars[i].speed = 10  # Speed Up
            else:
                cars[i].speed = 0  # stop
            timess[0].append(time.time() - outtime)
            # Check If Car Is Still Alive
            # Increase Fitness If Yes And Break Loop If Not
            updtime = time.time()
            cars[i].update(game_map)
            timess[1].append(time.time() - updtime)

            ups.append(cars[i].times)
            cars[i].times = 0
            # fitness[i] +=  cars[i].get_reward()

        # print("acts "+str(time.time()-acts))
        # Draw Map And All Cars That Are Alive
        screen.blit(game_map, (0, 0))
        drawtime = time.time()
        for i in range(len(agents)):
            cars[i].draw(screen)
        timess[2].append(time.time() - drawtime)
        pygame.display.flip()
        clock.tick(24 if render else 100000)
    # print("actions time " + str(sum(timess[0])))
    # print("update time " + str(sum(timess[1])))
    # print("draws time " + str(sum(timess[2])))
    # print("ups "+str(sum(ups))+"   "+str(len(ups)))
    # print("  alla cosa "+str(time.time()-ss))

    pygame.display.quit()
    dis = [
        math.sqrt((1730 - cars[i].center[0]) ** 2 + (165 - cars[i].center[1]) ** 2)
        for i in range(len(agents))
    ]
    # print(dis)
    return dis, obs

In [ ]:
import os.path
from typing import Any, Callable
import os
import inspyred.ec as ec
from random import Random

os.environ["PYGAME_HIDE_SUPPORT_PROMPT"] = "hide"


def readConfigFile(file: str) -> dict[Any, Any]:
    myvars = {}
    with open(file) as f:
        lines = f.read().splitlines()
        for line in lines:
            if line.startswith("#"):
                pass
            else:
                if "=" in line:
                    name, var = line.partition("=")[::2]
                    myvars[name.strip()] = var.strip()
    return myvars


def writeCandidatesToFile(file: str, candidates: list[list[float]]):
    with open(file, "w") as f:
        for candidate in candidates:
            for i in np.arange(len(candidate) - 1):
                f.write(str(candidate[i]) + " ")
            f.write(str(candidate[i]) + "\n")  # type: ignore


def readFileAsMatrix(file: str) -> list[list[float]]:
    with open(file) as f:
        lines = f.read().splitlines()
        matrix = []
        for line in lines:
            row = []
            for value in line.split():
                row.append(float(value.replace(",", ".")))
            matrix.append(row)
        return matrix


def fitness_eval(
    distanceToTarget: float,
    pathLength: float,
    noOfTimestepsWithCollisions: int,
    timestepToReachTarget: int,
    timestepsOnTarget: int,
) -> float:
    fitness = distanceToTarget
    return fitness


def eval(
    cs: list[list[float]], map: str, config: dict[str, Any], render: bool = False
) -> tuple[list[float], list[list[float]]]:
    sensors = 0 if not config["sensors"] else 4
    nrInputNodes = 2 + sensors  # nrIRSensors + nrDistanceSensor + nrBearingSensor
    nrHiddenNodes = int(config["nrHiddenNodes"])
    nrHiddenLayers = int(config["nrHiddenLayers"])
    nrOutputNodes = 5  # 2

    agents = [
        NN([
            nrInputNodes,
            *[nrHiddenNodes for _ in range(nrHiddenLayers)],
            nrOutputNodes,
        ])
        for _ in range(len(cs))
    ]
    for i in range(len(cs)):
        agents[i].set_weights(cs[i])
    dis, obs = run_simulation(agents, map=map, render=render)

    return dis, obs


#  this object is used for single-thread evaluations (only pickleable objects can be used in multi-thread)
class RobotEvaluator:
    def __init__(
        self,
        config: dict[str, Any],
        seed: int,
        eval_func: Callable[[float, float, int, int, int], float],
        maximize: bool,
    ):
        self.config = config

        sensors = 0 if not config["sensors"] else 4
        nrInputNodes = 2 + sensors  # nrIRSensors + nrDistanceSensor + nrBearingSensor
        nrHiddenNodes = int(config["nrHiddenNodes"])
        nrHiddenLayers = int(config["nrHiddenLayers"])
        nrOutputNodes = 5  # 2

        # calculate the no. of weights
        fka = NN([
            nrInputNodes,
            *[nrHiddenNodes for _ in range(nrHiddenLayers)],
            nrOutputNodes,
        ])

        nrWeights = fka.nweights

        self.geneMin = -3.0  # float(parameters["geneMin"])
        self.geneMax = 3.0  # float(parameters["geneMax"])
        self.nrTimeStepsGen = 0  # int(parameters["nrTimeStepsGen"])
        self.fitness_evaluator = eval_func
        self.nrWeights = nrWeights
        self.seed = seed
        self.bounder = ec.Bounder(
            [self.geneMin] * self.nrWeights, [self.geneMax] * self.nrWeights
        )
        self.maximize = maximize

        self.genCount = 0

    def generator(self, random: Random, args: dict[str, Any]) -> list[float]:
        return [
            random.uniform(self.geneMin, self.geneMax) for _ in range(self.nrWeights)
        ]

    def _distance(self, p1: list[float], p2: list[float]) -> float:
        return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

    def calcPathLength(self, obs: list[list[float]]) -> float:
        pathLength = 0
        for i in range(0, len(obs) - 1):
            pathLength += self._distance(obs[i][:2], obs[i + 1][:2])
        return pathLength

    def calcStepWithCollisions(self, obs: list[list[float]], eps: float = 10e-5) -> int:
        count = 0
        for i in range(len(obs)):
            if max(obs[i][2:]) <= eps:
                count += 1
        return count

    def timeStepToReachTarget(self, obs: list[list[float]]) -> int:
        count = 250
        for i in range(len(obs)):
            if self._distance(obs[i][:2], [1730, 165]) < 32:
                return i
        return count

    def timeOnTarget(self, obs: list[list[float]]) -> int:
        count = 0
        for i in range(len(obs)):
            if self._distance(obs[i][:2], [1730, 165]) < 32:
                count += 1
        return count

    def evaluator(
        self, candidates: list[list[float]], args: dict[str, Any]
    ) -> list[float]:
        # times = []
        results = []
        # s = time.time()
        results, observations = eval(
            candidates, "img/" + self.config["map"], self.config, False
        )
        fitness = []
        for i in np.arange(len(candidates)):
            distanceToTarget = results[i]
            pathLength = self.calcPathLength(observations[i])
            noOfTimestepsWithCollisions = self.calcStepWithCollisions(observations[i])
            timestepToReachTarget = self.timeStepToReachTarget(observations[i])
            timestepsOnTarget = self.timeOnTarget(observations[i])
            fitness_i = self.fitness_evaluator(
                distanceToTarget,
                pathLength,
                noOfTimestepsWithCollisions,
                timestepToReachTarget,
                timestepsOnTarget,
            )
            fitness.append(fitness_i)

        self.genCount += 1
        return fitness

In [ ]:
import pickle


def post_eval(pickle_path: str) -> tuple[list[float], list[list[float]]]:
    inputs = pickle.load(open(pickle_path, "rb"))
    return eval(inputs[0], inputs[1], inputs[2], True)

# Exercise 1
In this exercise we will perform an Evolutionary Robotics experiment to evolve the controller of a robot navigating a certain environment. This task is usually referred to as “maze navigation”. In principle, this kind of experiments can be done in hardware, i.e. with physical robots and a physical arena (this kind of approach is usually called embodied evolution). While embodied evolution is a quite powerful approach, it also presents several challenges due to limited battery lifetime, hardware issues/faults, costs, time-consuming experiments, etc. Here, for simplicity we will perform similar experiments in silico, i.e. by means of agent-based simulations. Note however that this is actually the typical approach used in Evolutionary Robotics, where usually simulations are performed first, and then the simulated experiments are replicated in materio (i.e. with physical hardware), to assess the so-called “reality-gap” and validate the solution.

Here, we will perform simplified 2-D kinematic simulations of a maze navigation task performed by an agent represented by a car.

The task is to drive from the bottom left of the map to the green zone in the upper right corner.
The car is equipped with 6 sensors: distance and bearing to the target position and, optionally, 4 lidar sensors. 
The first two return the distance and the angle between the car and the target. The lidars return the distance between the car and a possible obstacle in the range $[0,1]$. Hence, a value of $1$ indicates that there are no obstacles in the range of the lidar ($100$ pixels).
Instead, if a lidar returns $0$ it indicates that the car is colliding with an obstacle. 

A Feed Forward Neural Network (FFNN) moves the car on the map. The FFNN has $2$ or $6$ inputs, depending on if the lidars are activated, and $5$ outputs. The agent selects the action by the argmax of the outputs, and the possible actions are: turn left or right, accelerate or decelerate, and stop the car. 
The hidden structure of the network is composed of $l$ layers of $n$ nodes, where $l$ and $n$ are configurable parameters. 

The evolutionary algorithm has to optimize the weights of the FFNN, and the total number of parameters is $inputs \times n + l \times n^2 + n \times outputs$ if $n>=1$ else $inputs \times  outputs$. 
All the network’s weights range in $[−3, 3]$.

This script will perform a maze navigation Evolutionary Robotics experiment in a first scenario (environment without obstacles). This experiment uses an Evolutionary Algorithm to evolve the synaptic weights of the described Neural Network. The synaptic weights, which represent the genes of the individuals, are coded using real-coded values (floating point). A population of such individuals is evolved, using tournament selection, Gaussian mutation, n-point crossover,
and elitism (see exercise_maze.py for more details about the algorithmic configuration and parametrization). The genotypes of the first generation are initialized randomly in the range $[-3,3]$. Elitism is used to make sure that good solutions are not lost because of mutation or crossover, while the rest of the population is generated by means of the genetic operators. At each generation, the genotypes (i.e., the encoded controllers) generated by the EA are sent to the simulator (one batch per generation), which then translates the genotypes it receives into a set of Neural Network controllers, evaluates the controllers and sends the fitness back to the EA. 
At the end of the evolutionary process, the script will save a pickle containg the best genome (file best_seed.pkl, where seed is the seed used) found, and other information needed to run the simulation a posteriori.
You can “replay” the best candidate, by running this command:

``python3 post_eval.py best.pkl``

where ``best.pkl`` is the path to the pickle file.

---

Design and implement a fitness function that would allow the robot to reach the target as fast as possible. To do so, you can modify the function ``fitness_eval`` coded in the next cell. Five metrics are computed by the simulator and can be used to devise different fitness functions, namely:
- *distanceToTarget*: the distance to target at the end of the simulation (default);
- *pathLength*: the total distance traveled by the robot;
- *noOfTimestepsWithCollisions*: the no. of timesteps when the robot had a collision with an obstacle or a wall (ranging in [0,250]);
- *timestepToReachTarget*: the no. of timesteps needed for the robot to reach the target (ranging in [0,250]);
- *timestepsOnTarget*: the no. of timesteps spent by the robot on the target (ranging in [0,250]);

By using one or more of these five quantities properly into one single fitness value, different robot behaviors can be evolved. Please note that, by default, the RobotEvaluator is formulated as a minimization problem (see the flag ``maximize = False``). However, if you deem it more appropriate, you can change the flag to turn it into a maximization problem. 
**NOTE**: While you design the fitness function, be careful about divisions by zero and make sure that when you divide one integer variable by another you cast your variables to float, to avoid unexpected behaviors due to integer divisions.

- Is the Evolutionary Algorithm able to evolve a Neural Network controller that can reach the target? What kind of motion strategy does it use?
- What is the minimum-complexity Neural Network controller that you can think of? Hint: think about the necessity of using all the available sensor inputs in this case, and if any of them can be discarded (see the configuration dictionary to disable inputs). Also, consider reducing the no. if hidden nodes and test different network configurations to identify the simplest controller.
- By looking at the weights of the best evolved Neural Network in the simplest case you just found, can you try to make sense of the controller functioning? (Note that weights appear ordered by layer and, for each layer, by node.)

In [ ]:
# !pip3 install pygame
from utils.plot_utils import plot_observer
from utils.inspyred_utils import NumpyRandomWrapper
import os
from inspyred.ec import (
    terminators,
    observers,
    selectors,
    replacers,
    variators,
    EvolutionaryComputation,
)

# --------------------------------------------------------------------------- #
# Change this part, but not the map


def fitness_eval(  # noqa: F811
    distanceToTarget: float,
    pathLength: float,
    noOfTimestepsWithCollisions: int,
    timestepToReachTarget: int,
    timestepsOnTarget: int,
):
    print(
        distanceToTarget,
        pathLength,
        timestepToReachTarget,
        timestepsOnTarget,
        noOfTimestepsWithCollisions,
    )

    fitness = (
        float(distanceToTarget) / 1673.0 + float(noOfTimestepsWithCollisions) / 250.0
    )
    return fitness


config = {"sensors": True, "nrHiddenNodes": 5, "nrHiddenLayers": 1, "map": "white.png"}
seed = 0
rng = NumpyRandomWrapper(seed)

# --------------------------------------------------------------------------- #
# EA configuration
display = True

popSize = 10  # population size
numGen = 10  # used with generation_termination
numEval = 2500  # used with evaluation_termination
tournamentSize = 2  # tournament size (default 2)
mutationRate = 0.2  # mutation rate, per gene (default 0.1)
gaussianMean = 0  #  mean of the Gaussian distribution used for mutation
gaussianStdev = 0.1  #  std. dev. of the Gaussian distribution used for mutation
crossoverRate = 1.0  # rate at which crossover is performed (default 1.0)
numCrossoverPoints = 1  # number of crossover points used (default 1)
selectionSize = (
    popSize  # selection size (i.e. how many individuals are selected for reproduction)
)
numElites = 1  # no. of elites (i.e. best individuals that are kept in the population)

# the evolutionary algorithm (EvolutionaryComputation is a fully configurable evolutionary algorithm)
#  standard GA, ES, SA, DE, EDA, PAES, NSGA2, PSO and ACO are also available
ea = EvolutionaryComputation(rng)

# observers: provide various logging features
if display:
    ea.observer = [observers.stats_observer, plot_observer]  # type: ignore
    # observers.file_observer,
    # observers.best_observer,
    # observers.population_observer,

#  selection operator
# ea.selector = selectors.truncation_selection
# ea.selector = selectors.uniform_selection
# ea.selector = selectors.fitness_proportionate_selection
# ea.selector = selectors.rank_selection
ea.selector = selectors.tournament_selection

# variation operators (mutation/crossover)
ea.variator = [  # type: ignore
    variators.gaussian_mutation,
    variators.n_point_crossover,
]
# variators.random_reset_mutation,
# variators.inversion_mutation,
# variators.uniform_crossover,
# variators.partially_matched_crossover,

# replacement operator
# ea.replacer = replacers.truncation_replacement
# ea.replacer = replacers.steady_state_replacement
# ea.replacer = replacers.random_replacement
# ea.replacer = replacers.plus_replacement
# ea.replacer = replacers.comma_replacement
# ea.replacer = replacers.crowding_replacement
# ea.replacer = replacers.simulated_annealing_replacement
# ea.replacer = replacers.nsga_replacement
# ea.replacer = replacers.paes_replacement
ea.replacer = replacers.generational_replacement

# termination condition
# ea.terminator = terminators.evaluation_termination
# ea.terminator = terminators.no_improvement_termination
# ea.terminator = terminators.diversity_termination
# ea.terminator = terminators.time_termination
ea.terminator = terminators.generation_termination

# --------------------------------------------------------------------------- #

# the robot maze navigation problem
problem = RobotEvaluator(config, seed, eval_func=fitness_eval, maximize=False)

args = {}
args["fig_title"] = "EA"

# run the EA
final_pop = ea.evolve(
    generator=problem.generator,
    evaluator=problem.evaluator,
    bounder=problem.bounder,
    maximize=problem.maximize,
    pop_size=popSize,
    max_generations=numGen,
    # max_evaluations=numEval,
    tournament_size=tournamentSize,
    mutation_rate=mutationRate,
    gaussian_mean=gaussianMean,
    gaussian_stdev=gaussianStdev,
    crossover_rate=crossoverRate,
    num_crossover_points=numCrossoverPoints,
    num_selected=selectionSize,
    num_elites=numElites,
    **args,
)

# --------------------------------------------------------------------------- #
best_candidate = final_pop[0].candidate
best_fitness = final_pop[0].fitness
os.makedirs("results", exist_ok=True)
pickle.dump(
    ([best_candidate], f"img/{config['map']}", config, True),
    open("results/best_easy_" + str(seed) + ".pkl", "wb"),
)

In [ ]:
post_eval("results/best_easy_0.pkl")

# Exercise 2
Let us consider now a second scenario (environment with obstacles). To do so, change in the
configuration dict the parameter map to be ``hard.png``.
First of all, it is interesting to see if the controller evolved in the previous exercise is able to generalize its functioning to this case.
- Take the best Neural Network evolved in the previous exercise and run it in the new scenario, running the next cell.
- What happens in this case? Is the best Neural Network evolved in the previous exercise able to generalize to this new environment? Why?

In [ ]:
import pickle

best_path = f"results/best_easy_{seed}.pkl"
data = pickle.load(open(best_path, "rb"))
new_data = (data[0], "img/hard.png", data[2], True)
pickle.dump(new_data, open("results/tmp.pkl", "wb"))

post_eval("results/tmp.pkl")

Consider now running a new evolutionary process from scratch, to evolve a controller specific for this new, somehow harder environment.

- Is the same fitness function you designed in the previous exercise able to guide the evolutionary search also in this case? If not, try to change it appropriately (Hint: you may want to embed in the fitness value multiple metrics, and use some weights if needed). Does the best individual evolved in this scenario generalize to the first scenario?

- Try to make the problem even harder, in the attempt to find a controller that is able to drive the robot to the target without touching any walls. What kind of fitness function could you use in this case? You may also want to consider changing the configuration/parameters of the EA (e.g. larger population, higher number of generations, different mutation/crossover parameters) and/or the architecture/parameters of the Neural Network (e.g. add more hidden nodes).

In [ ]:
def fitness_eval(
    distanceToTarget: float,
    pathLength: float,
    noOfTimestepsWithCollisions: int,
    timestepToReachTarget: int,
    timestepsOnTarget: int,
):
    print(distanceToTarget, pathLength, timestepToReachTarget)
    fitness = (
        0.7 * float(distanceToTarget) / 1673.0
        + 0.4 * float(noOfTimestepsWithCollisions) / 250.0
    )
    return fitness


config = {"sensors": True, "nrHiddenNodes": 5, "nrHiddenLayers": 1, "map": "hard.png"}
seed = 0
rng = NumpyRandomWrapper(seed)

# --------------------------------------------------------------------------- #
# EA configuration
display = True

popSize = 20  # population size
numGen = 10  # used with generation_termination
numEval = 2500  # used with evaluation_termination
tournamentSize = 2  # tournament size (default 2)
mutationRate = 0.2  # mutation rate, per gene (default 0.1)
gaussianMean = 0  #  mean of the Gaussian distribution used for mutation
gaussianStdev = 0.1  #  std. dev. of the Gaussian distribution used for mutation
crossoverRate = 1.0  # rate at which crossover is performed (default 1.0)
numCrossoverPoints = 1  # number of crossover points used (default 1)
selectionSize = (
    popSize  # selection size (i.e. how many individuals are selected for reproduction)
)
numElites = 1  # no. of elites (i.e. best individuals that are kept in the population)

# the evolutionary algorithm (EvolutionaryComputation is a fully configurable evolutionary algorithm)
#  standard GA, ES, SA, DE, EDA, PAES, NSGA2, PSO and ACO are also available
ea = EvolutionaryComputation(rng)

# observers: provide various logging features
if display:
    ea.observer = [observers.stats_observer, plot_observer]  # type: ignore
    # observers.file_observer,
    # observers.best_observer,
    # observers.population_observer,

#  selection operator
# ea.selector = selectors.truncation_selection
# ea.selector = selectors.uniform_selection
# ea.selector = selectors.fitness_proportionate_selection
# ea.selector = selectors.rank_selection
ea.selector = selectors.tournament_selection

# variation operators (mutation/crossover)
ea.variator = [  # type: ignore
    variators.gaussian_mutation,
    variators.n_point_crossover,
]
# variators.random_reset_mutation,
# variators.inversion_mutation,
# variators.uniform_crossover,
# variators.partially_matched_crossover,

# replacement operator
# ea.replacer = replacers.truncation_replacement
# ea.replacer = replacers.steady_state_replacement
# ea.replacer = replacers.random_replacement
# ea.replacer = replacers.plus_replacement
# ea.replacer = replacers.comma_replacement
# ea.replacer = replacers.crowding_replacement
# ea.replacer = replacers.simulated_annealing_replacement
# ea.replacer = replacers.nsga_replacement
# ea.replacer = replacers.paes_replacement
ea.replacer = replacers.generational_replacement

# termination condition
# ea.terminator = terminators.evaluation_termination
# ea.terminator = terminators.no_improvement_termination
# ea.terminator = terminators.diversity_termination
# ea.terminator = terminators.time_termination
ea.terminator = terminators.generation_termination

# --------------------------------------------------------------------------- #

# the robot maze navigation problem
problem = RobotEvaluator(config, seed, eval_func=fitness_eval, maximize=False)

args = {}
args["fig_title"] = "EA"

# run the EA
final_pop = ea.evolve(
    generator=problem.generator,
    evaluator=problem.evaluator,
    bounder=problem.bounder,
    maximize=problem.maximize,
    pop_size=popSize,
    max_generations=numGen,
    # max_evaluations=numEval,
    tournament_size=tournamentSize,
    mutation_rate=mutationRate,
    gaussian_mean=gaussianMean,
    gaussian_stdev=gaussianStdev,
    crossover_rate=crossoverRate,
    num_crossover_points=numCrossoverPoints,
    num_selected=selectionSize,
    num_elites=numElites,
    **args,
)

# --------------------------------------------------------------------------- #
best_candidate = final_pop[0].candidate
best_fitness = final_pop[0].fitness
os.makedirs("results", exist_ok=True)
pickle.dump(
    ([best_candidate], f"img/{config['map']}", config, True),
    open("results/best_hard_" + str(seed) + ".pkl", "wb"),
)

In [ ]:
post_eval("results/best_hard_0.pkl")

Note, you can also design your map, just take the ``white.png`` in the utils folder and modify it with any image editor (e.g. gimp, paint, etc.). Remember to use Black to draw the walls and to make them thick enough, otherwise the car could pass through them. 
Save the new map in the same folder of the ``white.png`` file and modify the config dict element "map" with the name of your map.

## Instruction and questions
Concisely note down your observations from the previous exercises (follow the bullet points) and think about the following questions. 
- What do you think it could change between a simulated and a real-world experiment in the case of a maze navigation task?
- Can you think of some possible applications where a maze navigation robot task could be used? Why would it make sense to use Swarm/Evolutionary Robotics in those cases?